# Laboratorio 03: SQL

### Alejo Eyzaguirre

## Preparativos...

In [1]:
!wget https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release -O sqlite.tar.gz
!tar xzf sqlite.tar.gz
%cd sqlite/
!./configure
!make sqlite3.c
%cd /content
!npx degit coleifer/pysqlite3 -f
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build
!cp build/lib.linux-x86_64-3.6/pysqlite3/_sqlite3.cpython-36m-x86_64-linux-gnu.so \
    /usr/lib/python3.6/lib-dynload/_sqlite3.cpython-36m-x86_64-linux-gnu.so

--2020-10-18 22:33:02--  https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release
Resolving www.sqlite.org (www.sqlite.org)... 45.33.6.223, 2600:3c00::f03c:91ff:fe96:b959
Connecting to www.sqlite.org (www.sqlite.org)|45.33.6.223|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11075165 (11M) [application/x-compressed]
Saving to: ‘sqlite.tar.gz’

sqlite.tar.gz       100%[===================>]  10.56M  6.53MB/s    in 1.6s    

2020-10-18 22:33:05 (6.53 MB/s) - ‘sqlite.tar.gz’ saved [11075165/11075165]

/content/sqlite
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether gcc accepts -g... yes
chec

## Comienzo SQL

In [163]:
import sqlite3
sqlite3.sqlite_version 

'3.33.0'

In [164]:
!ls

build	       LICENSE	    README.md	 setup.py   sqlite3.h	   test
build-scripts  MANIFEST.in  sample_data  sqlite     sqlite.tar.gz  TODO
L03V2.db       pysqlite3    setup.cfg	 sqlite3.c  src


In [165]:
conexion = sqlite3.connect('L03V2.db')

cursorx = conexion.cursor()

In [246]:
cursorx.execute("DROP TABLE Continente")

In [247]:
cursorx.execute("DROP TABLE Regiones")

In [248]:
cursorx.execute("DROP TABLE Pais")

In [249]:
cursorx.execute("DROP TABLE Ecoinforma")

In [250]:
cursorx.execute("DROP TABLE District")

In [251]:
cursorx.execute("DROP TABLE Ciudad")

In [252]:
cursorx.execute("DROP TABLE Lenguajes")

In [253]:
cursorx.execute("DROP TABLE Lenguajeciudad")

In [254]:
cursorx.execute("DROP TABLE Lenguajepais")

## Misión 1: Preparando la Base de Datos:

En la siguiente sección se crean las distintas tablas que permitirán trabajar de manera más ordenada la base de datos, y por ende más eficientemente también. Desde ya se toma el camino simple de buscar la intersección ("join") entre ambas tablas `City` y `Eco`. Es decir, se consideran solo los países en que el nombre del país en la tabla city y el nombre del país en la tabla Eco son idénticos. Se toma este camino, dado que las tablas entregadas en la nueva base de datos sugieren que ese es el camino a seguir. La verdad es que lamento haber tenido que tomar este camino porque países como U.S.A quedan fuera, y hubiera sido bastante interesante agregar todos los países de los cuales tenemos información a los análisis que piden las consultas.

Sin embargo, para minimizar la pérdida de datos, a las demás tablas (fuera de `País` y la tabla `EcoInforma` (que contiene la data ecológica de cada país contenido en la tabla city)), se les *insertó* la información contenida en la tabla city. De esta forma, cuando se tengan que hacer consultas que no involucren hacer joins con la tabla Ecoinforma (es decir, entregar algún tipo de información ecológica) o con la tabla país, se abarcarán todos los países y ciudades contenidos en la tabla `city`.

In [255]:
cursorx.execute("CREATE TABLE Continente(continent TEXT PRIMARY KEY)")

In [256]:
cursorx.execute("CREATE TABLE Regiones(region TEXT PRIMARY KEY)")

In [257]:
cursorx.execute('''CREATE TABLE Pais(countrycode TEXT PRIMARY KEY, countryname TEXT, continent TEXT, region TEXT,
                countrysurface REAL, indepyear INTEGER, countrypopulation INTEGER, countrylifeexpectancy REAL, 
                gnp REAL, 
                gnpold REAL, countrylocalname TEXT, governmentform TEXT, headofstate TEXT, hdi REAL,  
                gdppercapita REAL)''')


In [258]:
cursorx.execute('''CREATE TABLE Ecoinforma(countrycode TEXT PRIMARY KEY, croplandfootprint REAL, 
                    grazingfootprint REAL, forestfootprint REAL, carbonfootprint REAL, 
                    fishfootprint REAL, totalecologicalfootprint REAL, cropland REAL, grazingland REAL, 
                    forestland REAL, fishingwater REAL, urbanland REAL, totalbiocapacity REAL, biocapacitydeficit REAL, 
                    dataquality REAL)''')


In [259]:
cursorx.execute('''CREATE TABLE District(did INTEGER PRIMARY KEY AUTOINCREMENT, district TEXT, countrycode TEXT,
                    FOREIGN KEY(countrycode) REFERENCES Pais)''')

In [260]:
cursorx.execute("CREATE TABLE Ciudad(id TEXT, name TEXT, countrycode TEXT, population INT, \
                did INTEGER, PRIMARY KEY(id), FOREIGN KEY(did) REFERENCES district, \
                FOREIGN KEY(countrycode) REFERENCES Pais)")


In [261]:
cursorx.execute("CREATE TABLE Lenguajes(lenguaje TEXT PRIMARY KEY)")



In [262]:
#Tablas que hacen puente a las relaciones N es a N.

cursorx.execute("CREATE TABLE Lenguajeciudad(id TEXT, language TEXT, \
                PRIMARY KEY(id, language), FOREIGN KEY(id) REFERENCES ciudad, FOREIGN KEY(language) REFERENCES Ciudad)")



In [263]:
cursorx.execute("CREATE TABLE Lenguajepais(countrycode TEXT, language TEXT, languagepercentage REAL, \
                islanguageofficial TEXT, \
                \
                PRIMARY KEY(countrycode, language), FOREIGN KEY(countrycode) REFERENCES Pais, \
                \
                FOREIGN KEY(language) REFERENCES city)")

conexion.commit()

#La siguiente tabla es equivalente a la tabla eco! no hay para que crearla...
#cursorx.execute("CREATE TABLE paiseco(country TEXT PRIMARY KEY, region TEXT, population TEXT, hdi TEXT, \
                #gdppercapita TEXT, croplandfootprint TEXT, grazingfootprint TEXT, forestfootprint TEXT, \
                #forestfootprint TEXT, totalecologicalfootprint TEXT, cropland TEXT, grazingland TEXT, \
                #forestland TEXT, urbanland TEXT, totalbiocapacity TEXT, biocapacitydeficit TEXT, \
                #dataquality TEXT)")
print(cursorx.fetchall())

[]


En la siguiente sección se Insertan los datos a las tablas ya creadas...

In [264]:
cursorx.execute('INSERT INTO Continente (continent) \
                \
                SELECT DISTINCT Continent \
                \
                FROM city')

In [265]:
cursorx.execute('INSERT INTO Regiones (region) \
                \
                SELECT DISTINCT Region \
                \
                FROM city')

In [266]:
conexion.commit()

Después de ver los datos, noté que el dato GDP PER CAPITA, incluye el signo peso. Por lo tanto, se procede a remplazar este valor en los datos de la columna que está en la tabla `eco`. El código para hacerlo lo saqué de la siguiente página.
https://stackoverflow.com/questions/14686204/how-to-replace-sql-field-value

Además para cambiar las comas por puntos, se usan los replace siguientes.

In [267]:
cursorx.execute('''UPDATE eco SET [gdp per capita] = REPLACE([gdp per capita], "$", "") ''')

In [268]:
cursorx.execute('''UPDATE eco SET [gdp per capita] = REPLACE([gdp per capita], "," , "|")''')

In [269]:
cursorx.execute('''UPDATE eco SET [gdp per capita] = REPLACE([gdp per capita], "," , ".")''')

In [270]:
cursorx.execute('''UPDATE eco SET [gdp per capita] = REPLACE([gdp per capita], "|" , "")''')

In [271]:
cursorx.execute('INSERT INTO Pais (countrycode, countryname, continent, region, \
                countrysurface, indepyear, countrypopulation, countrylifeexpectancy, gnp, \
                gnpold, countrylocalname, governmentform, headofstate, hdi,  \
                gdppercapita) \
                \
                SELECT DISTINCT C.countrycode, C.countryname, C.continent, C.region, \
                C.countrysurface, C.indepyear, C.countrypopulation, C.countrylifeexpectancy, C.gnp, \
                C.GNPOld, C.countrylocalname, C.governmentform, C.headofstate, E.hdi, E.[gdp per capita] \
                \
                FROM city C, eco E \
                \
                WHERE C.countryname = E.country \
                \
                GROUP BY C.countryname')

conexion.commit()

Corroboramos que los datos estén OK, en su formato indicado.

In [272]:
cursorx.execute('SELECT * FROM Pais')

conexion.commit()

print(cursorx.fetchall()[-10:])

[('TKM', 'Turkmenistan', 'Asia', 'Southern and Central Asia', 488100.0, 1991, 4459000, 60.9, 4397.0, 2000.0, 'Türkmenostan', 'Republic', 'Saparmurad Nijazov', 0.68, 5290.14), ('UGA', 'Uganda', 'Africa', 'Eastern Africa', 241038.0, 1962, 21778000, 42.9, 6313.0, 6887.0, 'Uganda', 'Republic', 'Yoweri Museveni', 0.48, 601.35), ('UKR', 'Ukraine', 'Europe', 'Eastern Europe', 603700.0, 1991, 50456000, 66.0, 42168.0, 49677.0, 'Ukrajina', 'Republic', 'Leonid Kutšma', 0.74, 3589.63), ('ARE', 'United Arab Emirates', 'Asia', 'Middle East', 83600.0, 1971, 2441000, 74.1, 37966.0, 36846.0, 'Al-Imarat al-´Arabiya al-Muttahida', 'Emirate Federation', 'Zayid bin Sultan al-Nahayan', 0.83, 40817.4), ('GBR', 'United Kingdom', 'Europe', 'British Islands', 242900.0, 1066, 59623400, 77.7, 1378330.0, 1296830.0, 'United Kingdom', 'Constitutional Monarchy', 'Elisabeth II', 0.9, 40980.5), ('URY', 'Uruguay', 'South America', 'South America', 175016.0, 1828, 3337000, 75.2, 20831.0, 19967.0, 'Uruguay', 'Republic', '

In [273]:
cursorx.execute('''INSERT INTO Ecoinforma (countrycode , croplandfootprint , 
                    grazingfootprint , forestfootprint , carbonfootprint , 
                    fishfootprint , totalecologicalfootprint , cropland , grazingland , 
                    forestland , fishingwater, urbanland , totalbiocapacity , biocapacitydeficit , 
                    dataquality ) 
                    
                    SELECT DISTINCT C.countrycode, E.[cropland footprint], E.[grazing footprint], E.[forest footprint], 
                    E.[carbon footprint], E.[fish footprint], E.[total ecological footprint], E.cropland, 
                    E.[grazing land], E.[forest land], E.[fishing water], E.[urban land], E.[total biocapacity], 
                    E.[biocapacity deficit or reserve], 
                    E.[data quality] 
                
                    FROM city C, eco E 
                
                    WHERE C.countryname = E.country 
                
                    GROUP BY C.countryname''')

In [274]:
cursorx.execute('SELECT * FROM Ecoinforma')

conexion.commit()

print(cursorx.fetchall()[-1:])

[('ZWE', 0.2, 0.32, 0.29, 0.53, 0.01, 1.37, 0.15, 0.32, 0.12, 0.01, 0.02, 0.62, -0.75, 6.0)]


In [275]:
cursorx.execute('SELECT P.countrycode, P.hdi FROM Pais P WHERE P.countryname = "Chile"')

conexion.commit()

print(cursorx.fetchall())

[('CHL', 0.83)]


In [276]:
cursorx.execute('''INSERT INTO district (district, countrycode)
    
                SELECT DISTINCT District, Countrycode
                
                FROM city''')

In [277]:
cursorx.execute('''SELECT * FROM District WHERE did = "337"''')
print(cursorx.fetchall())

[(337, 'Uttar Pradesh', 'IND')]


In [278]:
cursorx.execute('INSERT INTO ciudad (id, name, countrycode, population, did) \
                \
                SELECT DISTINCT C.ID, C.Name, C.CountryCode, C.Population, D.did \
                \
                FROM city C, District D \
                \
                WHERE D.district = C.district AND D.countrycode = C.countrycode')
                

In [279]:
cursorx.execute('SELECT * FROM ciudad WHERE id = 1234')

conexion.commit()

print(cursorx.fetchall())

[('1234', 'Jaunpur', 'IND', 136062, 337)]


Como se argumentó previamente, países como Estados Unidos debieron quedar fuera, por la diferencia de nombres. Si bien se podría haber metido a mano, preferí no hacerlo para no tener resultados distintos al resto.

In [280]:
cursorx.execute('SELECT * FROM Pais P WHERE P.countryname = "United States"')
conexion.commit()
print(cursorx.fetchall())

[]


In [281]:
cursorx.execute('INSERT INTO lenguajes (lenguaje) \
                \
                SELECT DISTINCT Language \
                \
                FROM city')

conexion.commit()

In [282]:
cursorx.execute('INSERT INTO lenguajeciudad (id, language) \
                \
                SELECT DISTINCT ID, Language \
                \
                FROM city')

conexion.commit()

In [283]:
cursorx.execute('SELECT * FROM lenguajeciudad')

conexion.commit()

print(cursorx.fetchall())

[('1', 'Balochi'), ('1', 'Dari'), ('1', 'Pashto'), ('1', 'Turkmenian'), ('1', 'Uzbek'), ('2', 'Balochi'), ('2', 'Dari'), ('2', 'Pashto'), ('2', 'Turkmenian'), ('2', 'Uzbek'), ('3', 'Balochi'), ('3', 'Dari'), ('3', 'Pashto'), ('3', 'Turkmenian'), ('3', 'Uzbek'), ('4', 'Balochi'), ('4', 'Dari'), ('4', 'Pashto'), ('4', 'Turkmenian'), ('4', 'Uzbek'), ('5', 'Arabic'), ('5', 'Dutch'), ('5', 'Fries'), ('5', 'Turkish'), ('6', 'Arabic'), ('6', 'Dutch'), ('6', 'Fries'), ('6', 'Turkish'), ('7', 'Arabic'), ('7', 'Dutch'), ('7', 'Fries'), ('7', 'Turkish'), ('8', 'Arabic'), ('8', 'Dutch'), ('8', 'Fries'), ('8', 'Turkish'), ('9', 'Arabic'), ('9', 'Dutch'), ('9', 'Fries'), ('9', 'Turkish'), ('10', 'Arabic'), ('10', 'Dutch'), ('10', 'Fries'), ('10', 'Turkish'), ('11', 'Arabic'), ('11', 'Dutch'), ('11', 'Fries'), ('11', 'Turkish'), ('12', 'Arabic'), ('12', 'Dutch'), ('12', 'Fries'), ('12', 'Turkish'), ('13', 'Arabic'), ('13', 'Dutch'), ('13', 'Fries'), ('13', 'Turkish'), ('14', 'Arabic'), ('14', 'Dutch'

In [284]:
cursorx.execute('INSERT INTO lenguajepais (countrycode, language, languagepercentage, islanguageofficial) \
                \
                SELECT DISTINCT CountryCode, Language, LanguagePercentage, IsLanguageOfficial \
                \
                FROM city \
                \
                GROUP BY Countrycode, Language')

conexion.commit()

In [285]:
cursorx.execute('SELECT * FROM lenguajepais WHERE countrycode = "CHL"')

conexion.commit()

print(cursorx.fetchall())

[('CHL', 'Aimará', 0.5, 'F'), ('CHL', 'Araucan', 9.6, 'F'), ('CHL', 'Rapa nui', 0.2, 'F'), ('CHL', 'Spanish', 89.7, 'T')]


## Misión 2: Conteos

### Misión 2a:

En cuántos países se habla el idioma X

In [286]:


cursorx.execute('SELECT DISTINCT L.language, COUNT(P.countryname) FROM Pais P, lenguajepais L\
                WHERE P.countrycode = L.countrycode\
                GROUP BY L.language' )

conexion.commit()


print("   IDIOMA   |   Nº PAÍSES   ")

for idioma, num in cursorx.fetchall():
  print("{0: <25s} {1: <15d} ".format(idioma, num))

   IDIOMA   |   Nº PAÍSES   
Abhyasi                   1               
Acholi                    1               
Adja                      1               
Afar                      2               
Afrikaans                 2               
Aimará                    3               
Ainu                      1               
Aizo                      1               
Akan                      1               
Albaniana                 2               
Ambo                      1               
Amhara                    1               
Ane                       1               
Arabic                    27              
Arabic-French             1               
Arabic-French-English     1               
Araucan                   1               
Arawakan                  2               
Armenian                  5               
Asami                     2               
Assyrian                  1               
Avarian                   1               
Azerbaijani              

En cuántos países se habla el idioma X y este es además idioma oficial para cada uno de esos países.

In [287]:
#En cuántos países se habla el idioma X, y es idioma oficial.
cursorx.execute('SELECT DISTINCT L.language, COUNT(P.countryname) FROM Pais P, Lenguajepais L\
                WHERE P.countrycode = L.countrycode AND L.islanguageofficial = "T"\
                GROUP BY L.language' )

conexion.commit()


print("   IDIOMA   |   Nº PAÍSES   ")

for idioma, num in cursorx.fetchall():
  print("{0: <20s} {1: <15d} ".format(idioma, num))

   IDIOMA   |   Nº PAÍSES   
Afrikaans            1               
Aimará               2               
Albaniana            1               
Arabic               19              
Armenian             1               
Azerbaijani          1               
Belorussian          1               
Bengali              1               
Bulgariana           1               
Burmese              1               
Chichewa             1               
Chinese              2               
Comorian             1               
Czech                1               
Danish               1               
Dari                 1               
Dutch                3               
Dzongkha             1               
English              18              
Estonian             1               
Ewe                  1               
Finnish              1               
French               13              
Georgiana            1               
German               5               
Greek                

### Misión 2b:

En cuántos países se habla el idioma X, en que el porcentaje de uso de este es mayor a 40%.

In [288]:
cursorx.execute('SELECT DISTINCT L.language, COUNT(P.countryname) FROM Pais P, Lenguajepais L\
                WHERE P.countrycode = L.countrycode AND L.languagepercentage > 40 \
                GROUP BY L.language' )

conexion.commit()


print("   IDIOMA   |   Nº PAÍSES   ")

for idioma, num in cursorx.fetchall():
  print("{0: <20s} {1: <15d} ".format(idioma, num))

   IDIOMA   |   Nº PAÍSES   
Akan                 1               
Albaniana            1               
Arabic               15              
Armenian             1               
Azerbaijani          1               
Bajan                1               
Belorussian          1               
Bengali              1               
Bulgariana           1               
Burmese              1               
Chichewa             1               
Chinese              2               
Comorian             1               
Creole English       7               
Creole French        6               
Crioulo              1               
Czech                1               
Danish               1               
Dutch                2               
Dzongkha             1               
English              7               
Estonian             1               
Fang                 1               
Finnish              1               
French               2               
Georgiana            

En cuántos países se habla el idioma X, en que el porcentaje de uso del idioma X es mayor a 40%, y este es oficial en ese país.

In [289]:
cursorx.execute('SELECT DISTINCT L.language, COUNT(P.countryname) \
                \
                FROM Pais P, lenguajepais L \
                \
                WHERE P.countrycode = L.countrycode AND L.languagepercentage > 40 \
                AND L.islanguageofficial = "T" \
                \
                GROUP BY L.language' )

conexion.commit()

print("   IDIOMA   |   Nº PAÍSES   ")

for idioma, num in cursorx.fetchall():
  print("{0: <20s} {1: <15d} ".format(idioma, num))

   IDIOMA   |   Nº PAÍSES   
Albaniana            1               
Arabic               15              
Armenian             1               
Azerbaijani          1               
Belorussian          1               
Bengali              1               
Bulgariana           1               
Burmese              1               
Chichewa             1               
Chinese              2               
Comorian             1               
Czech                1               
Danish               1               
Dutch                2               
Dzongkha             1               
English              6               
Estonian             1               
Finnish              1               
French               2               
Georgiana            1               
German               3               
Greek                2               
Guaraní              1               
Hebrew               1               
Hungarian            1               
Italian              

### Misión 2c: 

¿Cu ́antospa ́ısestienenunporcentajemayora10%desusciudadesconm ́asde1mill ́ondehabitantes?

In [290]:
cursorx.execute('''SELECT COUNT(*) 
                    FROM (SELECT P.countrycode, COUNT(C.id) AS contador FROM ciudad C, Pais P 
                    WHERE C.population > 1000000 AND P.countrycode = C.countrycode GROUP BY P.countryname) G, 
                   (SELECT P.countrycode, COUNT(C.id)*1.0 AS conta FROM ciudad C, Pais P 
                    WHERE P.countrycode = C.countrycode GROUP BY P.countryname) C 
                    WHERE C.countrycode = G.countrycode AND ((G.contador*1.0) /(C.conta*1.0) > 0.1)''')

conexion.commit()


for i in cursorx.fetchall():
    print(i[0])

23


### Mision 2d: 
¿En cuántos países donde se hable 5 o más lenguajes, el porcentaje de suelo urbano es mayoritario?



In [291]:
cursorx.execute('''SELECT COUNT(*)
                
                FROM (SELECT * FROM (SELECT L.countrycode, COUNT(L.language) 
                                     AS contemos FROM Lenguajepais L GROUP BY L.countrycode) Y 
                               WHERE Y.contemos > 5) J,
                    
                    (SELECT P.countrycode , E.urbanland 
                    FROM Ecoinforma E, Pais P 
                    WHERE E.countrycode = P.countrycode 
                    AND E.urbanland != "" 
                    AND E.urbanland > (E.cropland + E.forestland + E.grazingland + E.fishingwater)) Z
                    
                WHERE J.countrycode = Z.countrycode''')


for i in cursorx.fetchall():
    print(i[0])

0


## Misión 3: Encontrando Registro Llamativos

### Misión 3a:

Encuentre el país más rico de la región más pobre del mundo. Para la siguiente consulta, se considera como la región oficial del país, a la región entregada en la tabla `city`. Por lo tanto el listado de regiones se detalla en la siguiente consulta.

In [292]:
cursorx.execute("SELECT DISTINCT P.region FROM Pais P")

for i in cursorx.fetchall():
    print(i[0])

Southern and Central Asia
Southern Europe
Northern Africa
Central Africa
Caribbean
South America
Middle East
Australia and New Zealand
Western Europe
Eastern Europe
Western Africa
North America
Southern Africa
Eastern Africa
Southeast Asia
Eastern Asia
Central America
Nordic Countries
Baltic Countries
Polynesia
British Islands
Micronesia
Melanesia


Como criterio de pobreza, se estima que el **GNP per capita**, es una buena representación de la pobreza del país. Se considera esta variable dado que el GNP es el Producto Nacional Bruto, es decir el PIB menos el Pago neto de factores. En pocas palabras el PNB, corresponde al Producto Generado por los Residentes de un país y no el producto generado dentro del terreno geográfico del país (PIB). 

El hecho de que sea per cápita, significa que representa el PNB por habitante del país. Esto significa que el PNB se divide por la población y se utiliza como proxy de la pobreza local, ya que tenemos países con grandes valores de PNB, pero con mucha población por lo cual el PNB por si solo, puede no ser una buena medida de pobreza. 

Es importante tener en cuenta que el PNB per cápita no es una medida perfecta de pobreza, ya que no considera la desigualdad económica que puede haber dentro del país. Quizás el PNB es solo percibido por unos pocos, por lo que un mayor valor de PNB per cápita puede también no ser una buena proxy de pobreza.

La región más pobre será la región que tenga el GNP per cápita promedio (entre todos los países de la región) menor. En el siguiente cuadro se calcula:

In [293]:
cursorx.execute('''SELECT P.region, AVG(P.gnp / P.countrypopulation) AS gnppercapita 
                    FROM Pais P 
                    GROUP BY P.region 
                    ORDER BY gnppercapita ASC        
                ''')

print(cursorx.fetchall())

[('Western Africa', 0.0003366370300772401), ('Southern and Central Asia', 0.0004360144321088999), ('Central Africa', 0.0009295092817698098), ('Eastern Africa', 0.0016569809398328096), ('Southern Africa', 0.001871423550106314), ('Polynesia', 0.0019129772906368652), ('Central America', 0.0021013766318796674), ('Eastern Europe', 0.002550252007473336), ('Northern Africa', 0.0028306718127649516), ('Baltic Countries', 0.00307739320048289), ('South America', 0.003378226982507794), ('Southeast Asia', 0.0049899119464790745), ('Melanesia', 0.006032365347427689), ('Middle East', 0.006838875769545767), ('Caribbean', 0.007318502254468636), ('Southern Europe', 0.009040949693603167), ('Eastern Asia', 0.010349068755474105), ('Australia and New Zealand', 0.016375225500568073), ('Micronesia', 0.016416666666666666), ('British Islands', 0.021614128238591414), ('North America', 0.02752117033125798), ('Western Europe', 0.028341016435982105), ('Nordic Countries', 0.02859381050638741)]


En el siguiente cuadro se calculan los países con su respectivo GNP per cápita. Se ordena esta tabla de país más pobre a menos pobre para entender mejor la base de datos. 

In [294]:
cursorx.execute('''SELECT P.countrycode, P.region, (P.gnp / P.countrypopulation) AS gnppercapita 
                    FROM Pais P 
                    GROUP BY P.countrycode 
                    ORDER BY gnppercapita ASC
                
                ''')

print(cursorx.fetchall())

[('STP', 'Central Africa', 4.0816326530612245e-05), ('SOM', 'Eastern Africa', 9.260176289987125e-05), ('ETH', 'Eastern Africa', 0.00010154239590825541), ('BDI', 'Eastern Africa', 0.00013487677371172517), ('MOZ', 'Eastern Africa', 0.00014690040650406505), ('SLE', 'Western Africa', 0.00015368768026370003), ('MWI', 'Eastern Africa', 0.00015441647597254005), ('TCD', 'Central Africa', 0.0001578878577963665), ('NER', 'Western Africa', 0.00015899347623485555), ('ERI', 'Eastern Africa', 0.00016883116883116884), ('BTN', 'Southern and Central Asia', 0.0001751412429378531), ('NPL', 'Southern and Central Asia', 0.00019924780610112828), ('BFA', 'Western Africa', 0.0002031498701516294), ('MLI', 'Western Africa', 0.000235178921132277), ('MDG', 'Eastern Africa', 0.00023522770041400074), ('GNB', 'Western Africa', 0.00024154987633965376), ('GMB', 'Western Africa', 0.00024521072796934864), ('BGD', 'Southern and Central Asia', 0.00025436103906159265), ('AFG', 'Southern and Central Asia', 0.000263028169014

In [295]:
cursorx.execute('''SELECT Z.countryname
                    FROM (SELECT P.region, AVG(P.gnp / P.countrypopulation) AS gnppercapita 
                        FROM Pais P 
                        GROUP BY P.region 
                        ORDER BY gnppercapita ASC LIMIT 1) Y,
                         (SELECT P.countrycode, P.countryname, P.region, (P.gnp / P.countrypopulation) 
                         AS gnppercapita 
                        FROM Pais P 
                        GROUP BY P.countrycode 
                        ORDER BY gnppercapita ASC) Z
                    WHERE Y.region = Z.region
                    GROUP BY Z.countrycode
                    ORDER BY Z.gnppercapita ASC
                    LIMIT 1
                ''')

for i in cursorx.fetchall():
    print(i[0])

Sierra Leone


### Misión 3b:

Para la región más pobre del mundo, indique el porcentaje de su suelo que está cubierto por bosques. En esta cosulta se considera como región la región de un país, la región entregada en la tabla city y no eco. Se utiliza nuevamente el GNP per Capita como medida de pobreza.

Además al pedir el **porcentaje** de suelo cubierto por bosques, se entrega el porcentaje promedio de la región más pobre.

In [296]:
cursorx.execute('''SELECT Y.region, (ROUND(AVG(E.forestland), 4) *100)
                    FROM (SELECT P.region, AVG(P.gnp / P.countrypopulation) AS gnppercapita 
                        FROM Pais P 
                        GROUP BY P.region 
                        ORDER BY gnppercapita ASC LIMIT 1) Y,
                        
                        (SELECT P.countrycode, P.countryname, P.region, (P.gnp / P.countrypopulation) 
                         AS gnppercapita 
                        FROM Pais P 
                        GROUP BY P.countrycode 
                        ORDER BY gnppercapita ASC) Z,
                        Ecoinforma E
                    WHERE E.countrycode = Z.countrycode AND Z.region = Y.region
                    GROUP BY Y.region
                    ORDER BY Y.gnppercapita ASC
                ''')

for i in cursorx.fetchall():
    print("Región: " + i[0] + " | Porcentaje de suelo cubierto por bosques: " + str(i[1])+"%")

Región: Western Africa | Porcentaje de suelo cubierto por bosques: 32.57%


### Misión 3c:

Para los 5 idiomas más hablados del mundo (número de personas que hablan ese idioma). Identifique para cada idioma, el país donde ese idioma es oficial y predominante que deje la mayor huella ecológica. Ordene los países resultantes en orden decreciente de población.

En la siguiente consulta se dividió nuevamente el problema en pequeños subproblemas. El primero a atacar fue:

Encontrar los 5 idiomas más hablados del mundo (cantidad de población que los habla).

In [297]:
cursorx.execute('''SELECT L.language, SUM(P.countrypopulation*(L.languagepercentage/100)) AS sumapob
                    FROM Pais P, Lenguajepais L
                    WHERE P.countrycode = L.countrycode
                    GROUP BY L.language
                    Order by sumapob DESC
                    Limit 5''')


for i in cursorx.fetchall():
    print(i)

('Chinese', 1188948520.0)
('Hindi', 405276040.99999994)
('Spanish', 308623847.0)
('Bengali', 209304719.0)
('Arabic', 200000525.7)


El siguiente código lo usé para comprobar que los datos estén bien...!

Select DISTINCT L.language, P.countryname, P.countrypopulation, LP.languagepercentage,
P.countrypopulation * (LP.languagepercentage/100)
FROM Lenguajeciudad L, Ciudad C, Pais P, Lenguajepais LP
Where L.language = "Zulu" 
AND L.id = C.id 
And C.countrycode = P.countrycode 
AND LP.countrycode = P.countrycode
AND LP.language = L.language

Luego buscamos los lenguajes oficiales y predominantes por país, agregando a esta nueva tabla a entregar la huella ecológica que emite cada país.

Para considerar un lenguaje predominante en un país se consideró que el lenguaje era predominante si era el que más se hablaba dentro de un país. 

In [298]:
cursorx.execute('''SELECT L.countrycode, P.countryname, L.language, MAX(L.languagepercentage), 
                    E.totalecologicalfootprint
                    
                    FROM Lenguajepais L, Ecoinforma E, Pais P
                    
                    WHERE L.islanguageofficial = "T" 
                    AND E.countrycode = L.countrycode
                    AND L.countrycode = P.countrycode
                    AND E.carbonfootprint != ""
                    
                    GROUP BY L.countrycode
                    ORDER BY E.totalecologicalfootprint DESC''')


for i in cursorx.fetchall():
    print(i)

('LUX', 'Luxembourg', 'Luxembourgish', 64.4, 15.82)
('QAT', 'Qatar', 'Arabic', 40.7, 10.8)
('AUS', 'Australia', 'English', 81.2, 9.31)
('CAN', 'Canada', 'English', 60.4, 8.17)
('KWT', 'Kuwait', 'Arabic', 78.1, 8.13)
('SGP', 'Singapore', 'Chinese', 77.1, 7.97)
('ARE', 'United Arab Emirates', 'Arabic', 42.0, 7.93)
('OMN', 'Oman', 'Arabic', 76.7, 7.52)
('BHR', 'Bahrain', 'Arabic', 67.7, 7.49)
('BEL', 'Belgium', 'Dutch', 59.2, 7.44)
('SWE', 'Sweden', 'Swedish', 89.5, 7.25)
('LVA', 'Latvia', 'Latvian', 55.1, 6.29)
('ISR', 'Israel', 'Hebrew', 63.1, 6.22)
('MNG', 'Mongolia', 'Mongolian', 78.8, 6.08)
('AUT', 'Austria', 'German', 92.0, 6.06)
('LTU', 'Lithuania', 'Lithuanian', 81.6, 5.83)
('SVN', 'Slovenia', 'Slovene', 87.9, 5.81)
('CHE', 'Switzerland', 'German', 63.6, 5.79)
('RUS', 'Russian Federation', 'Russian', 86.6, 5.69)
('CYM', 'Cayman Islands', 'English', 0.0, 5.65)
('SAU', 'Saudi Arabia', 'Arabic', 95.0, 5.61)
('NZL', 'New Zealand', 'English', 87.0, 5.6)
('IRL', 'Ireland', 'English', 98

Juntando las dos consultas realizadas previamente y agregando unos ajustes, armé la consulta final (perdón lo larga...)

In [299]:
cursorx.execute('''SELECT N.countryname FROM (
                    
                    
                    SELECT Z.countryname, MAX(Z.totalecologicalfootprint), Y.countrypopulation FROM 
                        
                        (SELECT L.language, countrypopulation, 
                        SUM(P.countrypopulation*(L.languagepercentage/100)) AS sumapob
                        FROM Pais P, Lenguajepais L
                        WHERE P.countrycode = L.countrycode
                        GROUP BY L.language
                        ORDER BY sumapob DESC
                        Limit 5) Y,
                        
                        (SELECT L.countrycode, P.countryname, L.language, MAX(L.languagepercentage), 
                        E.totalecologicalfootprint
                        FROM Lenguajepais L, Ecoinforma E, Pais P
                        WHERE L.islanguageofficial = "T" 
                         AND E.countrycode = L.countrycode
                         AND P.countrycode = L.countrycode
                         AND E.totalecologicalfootprint != ""
                        GROUP BY L.countrycode
                        ORDER BY E.totalecologicalfootprint DESC) Z
                        
                        WHERE Y.language = Z.Language
                        GROUP BY Y.language
                        ORDER BY Z.totalecologicalfootprint DESC) N

                  ORDER BY N.countrypopulation DESC''')

conexion.commit()

contador = 1
print("Los 5 países son:")
for i in cursorx.fetchall():
    print(str(contador) + ".- " + i[0])
    contador += 1 

Los 5 países son:
1.- Bangladesh
2.- Singapore
3.- Qatar
4.- India
5.- Chile


### Misión 3d:

Asumiendo que el uso de un lenguaje distribuye de manera homogénea en cada país, encuentre la ciudad del mundo donde hay menos personas hablando un lenguaje, con la condición que exista al menos una. Por cada ciudad encontrada (si hay más de una), indique además de la ciudad, el lenguaje, si es oficial y la cantidad de personas que lo hablan.

Ojo que acá como no se usa la tabla `País` o `Ecoinforma`, también se consideran ciudades en que sus países no están en las dos tablas mencionadas.

In [300]:
cursorx.execute('''SELECT C.name, LC.language, LP.islanguageofficial,
                    ROUND(C.population * (LP.languagepercentage/100)) AS cantidadpersonas 
                    FROM Lenguajeciudad LC, Ciudad C, Lenguajepais LP 
                    WHERE LC.id = C.id AND LP.Language = LC.language  AND LP.countrycode = C.countrycode
                    AND cantidadpersonas > 0
                    ORDER BY cantidadpersonas ASC
                    LIMIT 1''')

for ciudad, pais, oficial, poblacion in cursorx.fetchall():
    print(" Ciudad: {} \n Idioma: {} \n ¿Es idioma oficial?: {} \n Gente que lo habla: {}".format(ciudad, pais, oficial, poblacion))

 Ciudad: Bairiki 
 Idioma: Tuvalu 
 ¿Es idioma oficial?: F 
 Gente que lo habla: 11.0


## Misión 4: Rankings



### Misión 4a:

Encuentre los 10 distritos con mayor y menor porcentaje de población, en relación al país al que pertenecen.

En esta misión se interpretó que se debía *ranquear* los distritos por la población que tienen estos, relativa a la población total del país al que pertenecen.

Es necesario agrupar la consulta DID, el ID de cada Distrito, porque pueden haber Distritos con igual nombre. Además no se considera el dato en que el distrito se llama "-" y viene de Singapur, dado que ese dato presenta una incoherencia (población del distrito > población del país). No se considera en el ranking, los distritos "-" de un cierto país, ya que al no tener nombre ni algún tipo de identificador, estos debieran ser un dato vacío, o alguna anomalía del país que distorsionaría los datos de respuesta.

In [301]:
#Selecciono distritos y veo el porcentaje relativo de población de estos con respecto a su país.

cursorx.execute('''SELECT Z.district FROM(
                    SELECT * , (ROW_NUMBER() OVER (ORDER BY K.fraccion DESC)) AS ranking
                    FROM(
                    SELECT C.did, D.district, C.countrycode, (SUM(C.population*1.0)/(P.countrypopulation*1.0)) AS fraccion
                    FROM Ciudad C, Pais P, District D
                    WHERE C.countrycode = P.countrycode 
                    AND D.countrycode = P.countrycode 
                    AND C.did = D.did 
                    AND D.district != "–"
                    GROUP BY D.did
                    HAVING fraccion < 1.0
                    ORDER BY fraccion DESC) K) Z
                    WHERE Z.ranking < 11 OR Z.ranking > 1103''')



contador = 0
for i in cursorx.fetchall():
    if contador == 0:
      print("Distritos MÁS Poblados Relativo a la población total de su país\n") 
    
    if contador == 10:
      print("\nDistritos MENOS Poblados Relativo a la población total de su país\n") 
    
    contador += 1
    print(i[0])  

   


Distritos MÁS Poblados Relativo a la población total de su país

Djibouti
Doha
New Providence
Grand Cayman
Montevideo
Santiago
Yerevan
St John
Estuaire
Aqua Grande

Distritos MENOS Poblados Relativo a la población total de su país

Jersey
Sulawesi Tenggara
West Irian
Hainan
Pondicherry
Manipur
Tripura
Mizoram
Meghalaya
Tibet


### Misión 4b:

Construya un ranking descendente de las regiones del mundo más contaminantes, donde para cada una de ellas se indique en orden ascendente los 3 países más contaminantes.

En esta pregunta se consideró la Huella de Carbono, como la medida de contaminación de un país. Por lo tanto, la región más contaminante sería la que la mayor suma de la huella de carbono de los países que están dentro de ella.

Para construir este ranking se utilizaron dos consultas centrales. La primera permitió tener un ranking de los 3 países más contaminantes por cada región. Para esta consulta se utilizó el comando `ROW_NUMBER()` que no me funcionaba en Jupyter Notebook así que tuve que cambiarme a Google Colab y además ejecutar el primer cuadro de código que tengo en este cuaderno. Luego de haber hecho esta consulta tenía los 3 países más contaminantes por región ordenados de manera Descendiente.

Luego se creó la segunda consulta, que entrega las regiones más contaminantes, pero su gran función es encontrar la suma de las huellas de carbono de los países contenidos dentro de cada región.

En el último paso se utlizan ambas consultas, y para lograr el orden pedido se hace lo siguiente: Para cada uno de los 3 países más contaminantes de cada región, se suma la suma de huellas de carbono total de la región, que le corresponde al país, con el ranking del país dentro de la región (entregado en la primera consulta) por un número arbitrariamente pequeño (0.001). Finalmente se ordena la consulta que envuelve ambas consultas por esta suma, obteniendo el ranking solicitado. 

Es importante tener presente que las regiones con menos de 3 países, es porque es región tenía menos países "válidos" que 3. 

In [302]:
  cursorx.execute('''SELECT ranking_region.region, ranking_pais_x_region.countryname FROM 

                    (SELECT * FROM (SELECT P.countryname , P.region, (E.carbonfootprint), 
                    ROW_NUMBER() OVER(PARTITION BY region ORDER BY carbonfootprint DESC) AS ranking
                    FROM Pais P, Ecoinforma E
                    WHERE P.countrycode = E.countrycode AND E.carbonfootprint != "") original
                    WHERE original.ranking < 4) ranking_pais_x_region,

                    (SELECT P.region , SUM(E.carbonfootprint) AS suma
                    FROM Pais P, Ecoinforma E
                    Where P.countrycode = E.countrycode
                    GROUP BY P.region
                    ORDER BY suma DESC) ranking_region

                    WHERE ranking_region.region = ranking_pais_x_region.region

                    ORDER BY (ranking_region.suma + ranking_pais_x_region.ranking*0.001) DESC''')
  
print("\tRegiones y sus países más contaminantes\n")
print("REGION                             PAÍS              \n")
for region, pais in cursorx.fetchall():
    print("{0: <25s} |{1: <25s} ".format(region, pais))



	Regiones y sus países más contaminantes

REGION                             PAÍS              

Middle East               |United Arab Emirates      
Middle East               |Kuwait                    
Middle East               |Qatar                     
Western Europe            |Switzerland               
Western Europe            |Belgium                   
Western Europe            |Luxembourg                
Caribbean                 |Cayman Islands            
Caribbean                 |Bahamas                   
Caribbean                 |Trinidad and Tobago       
Eastern Europe            |Slovakia                  
Eastern Europe            |Czech Republic            
Eastern Europe            |Russian Federation        
Southern Europe           |Greece                    
Southern Europe           |Italy                     
Southern Europe           |Slovenia                  
South America             |French Guiana             
South America             |Chile       

### Misión 4c:

Para las 5 regiones más ricas del mundo, ordenadas de manera ascendente, indique el porcentaje de cada uno de sus usos de suelo, ordenados de acuerdo a algún criterio definido por ud.

Para esta misión también se considera el GNP per cápita como medida de riqueza de una nación, por los argumentos dados en la `Misión 3a`. 

In [303]:
cursorx.execute('''SELECT suelos.region, ROUND(suelos.promcrop, 4), ROUND(suelos.promgrazing,4), 
                    ROUND(suelos.promforest, 4), ROUND(suelos.promfish, 4), ROUND(suelos.promurban, 4)
                    FROM
                    (SELECT P.region , AVG(E.cropland) AS promcrop,
                    AVG(E.grazingland) AS promgrazing,
                    AVG(E.forestland) AS promforest,
                    AVG(E.fishingwater) AS promfish,
                    AVG(E.urbanland) as promurban,
                    ((AVG(E.cropland) + AVG(E.grazingland) + AVG(E.forestland) + AVG(E.fishingwater) 
                    + AVG(E.urbanland))/4) AS promedio_uso
                    FROM Pais P, Ecoinforma E
                    Where P.countrycode = E.countrycode
                    GRoup by P.region
                    HAVING promedio_uso > 0
                    ORDER BY promedio_uso) suelos,

                    (SELECT P.region, AVG(P.gnp / P.countrypopulation) AS gnppercapita 
                    FROM Pais P 
                    GROUP BY P.region 
                    ORDER BY gnppercapita DESC
                    LIMIT 6) riqueza

                    WHERE riqueza.region = suelos.region
                    ORDER BY riqueza.gnppercapita ASC''')


print("\t\tUsos promedio de suelo por región\n")
print("REGION                       | CROPLAND | GRAZING | FOREST  | FISHING | URBAN  |")
for region, cropland, grazing, forest, fishing, urban in cursorx.fetchall():
    print("{0: <30s} {1: >4f} {2: f} {3: f} {4: f} {5: f}".format(region, cropland, grazing, forest, fishing, urban))


		Usos promedio de suelo por región

REGION                       | CROPLAND | GRAZING | FOREST  | FISHING | URBAN  |
Australia and New Zealand      2.965000  4.255000  3.980000  2.025000  0.135000
British Islands                0.720000  0.440000  0.310000  0.895000  0.155000
North America                  1.555000  0.145000  4.560000  1.705000  0.035000
Western Europe                 0.787100  0.092900  0.810000  0.094300  0.185700
Nordic Countries               0.922500  0.057500  1.750000  0.997500  0.122500


### Misión 4d:

Por cada una de las regiones del mundo, encuentre los 5 países donde las personas generan mayor “riqueza” durante sus vidas.

Se considera como la riqueza generada en la vida, el PIB per cápita multiplicado por la esperanza de vida, ya que el pib per cápita es la riqueza generada dentro del país, dividida por el total de individuos. Entonces este es como la riqueza que genera cada individuo en un año (PIB es un flujo) entonces si lo multiplicamos por los años que vive cada individuo, sería una proxy (no muy buena) de la riqueza generada en la vida del individuo.

In [304]:
cursorx.execute('''SELECT H.region, H.ranking, H.countryname FROM
                    (SELECT Z.countryname, Z.region, Z.plataenvida, 
                    ROW_NUMBER() OVER(PARTITION BY region ORDER BY Z.plataenvida DESC) AS ranking
                    FROM (SELECT P.countryname, P.region, 
                    ROUND((P.gdppercapita*P.countrylifeexpectancy),4) AS plataenvida
                    FROM Pais P) Z) H
                    WHERE H.ranking < 6
                    ''')

print("\tRegiones y sus países más contaminantes\n")
print("REGION                     Nº   PAÍS              \n")
for region, ranking, pais in cursorx.fetchall():
    print("{0: <25s} {1: d}.- {2: <25s} ".format(region, ranking, pais))

	Regiones y sus países más contaminantes

REGION                     Nº   PAÍS              

Australia and New Zealand  1.- Australia                 
Australia and New Zealand  2.- New Zealand               
Baltic Countries           1.- Estonia                   
Baltic Countries           2.- Lithuania                 
Baltic Countries           3.- Latvia                    
British Islands            1.- Ireland                   
British Islands            2.- United Kingdom            
Caribbean                  1.- Bahamas                   
Caribbean                  2.- Trinidad and Tobago       
Caribbean                  3.- Barbados                  
Caribbean                  4.- Antigua and Barbuda       
Caribbean                  5.- Saint Kitts and Nevis     
Central Africa             1.- Equatorial Guinea         
Central Africa             2.- Gabon                     
Central Africa             3.- Angola                    
Central Africa             4.- Congo

## Misión 5: Bonus



### Misión 5a:

Considere que las huellas ecológicas se distribuyen homogéneamente entre las ciudades de un país en base a sus poblaciones. Incorpore en su tabla de ciudades las huellas ecológicas desagregadas y la total.

NO ALCANCÉ A HACERLA, PERDÓN...!

### Misión 5b: 

Construya un ranking con las ciudades que más huella ecológica dejan por continente (máximo 3 ciudades por continente) solo para ciudades que sean parte de los 20 países que dejan la mayor huella ecológica.

In [305]:
cursorx.execute('''SELECT Z.continent, Z.ranking, Z.name FROM 
                  (SELECT C.name, P.continent, C.countrycode, 
                  ((E.totalecologicalfootprint * C.population)/P.countrypopulation),
                  ROW_NUMBER() OVER(PARTITION 
                  BY P.continent 
                  ORDER BY ((E.totalecologicalfootprint * C.population)/P.countrypopulation) DESC) 
                  AS ranking

                  FROM Ciudad C, Pais P, Ecoinforma E
                  WHERE C.countrycode = P.countrycode AND E.countrycode = P.countrycode) Z

                  WHERE Z.ranking < 4 AND Z.countrycode IN (SELECT F.countrycode FROM 
                                                          (SELECT P.countrycode, P.countryname, 
                                                          E.totalecologicalfootprint 
                                                          FROM Pais P, Ecoinforma E
                                                          WHERE P.countrycode = E.countrycode
                                                          Order by E.totalecologicalfootprint DESC
                                                          LIMIT 20) F)''')

print("\tRegiones y sus países más contaminantes\n")
print("Continente                 Nº   Ciudad              \n")
for region, ranking, pais in cursorx.fetchall():
    print("{0: <25s} {1: d}.- {2: <25s} ".format(region, ranking, pais))


	Regiones y sus países más contaminantes

Continente                 Nº   Ciudad              

Asia                       1.- Singapore                 
Asia                       2.- Doha                      
Asia                       3.- Dubai                     
Europe                     1.- Luxembourg [Luxemburg/Lëtzebuerg] 
Europe                     2.- Riga                      
Europe                     3.- Tallinn                   
North America              1.- Nassau                    
North America              2.- Oranjestad                
Oceania                    1.- Sydney                    
Oceania                    2.- Melbourne                 


In [306]:
conexion.commit()

In [307]:
conexion.close()